<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Dataset-Preparation-Notebook" data-toc-modified-id="Dataset-Preparation-Notebook-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Dataset Preparation Notebook</a></span><ul class="toc-item"><li><span><a href="#Developement" data-toc-modified-id="Developement-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Developement</a></span></li><li><span><a href="#Results" data-toc-modified-id="Results-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Results</a></span></li></ul></li></ul></div>

# Dataset Preparation Notebook
---
In this notebook we will prepare a dataset for training from the `.csv` data provided. We will use the following **tips** as the general guidline for developing this dataset:
 1. The first sentence from the therapist's message is often a good reflection.
 2. Sentences that start with "seems like" or sounds like" have the potential to be good reflections.

Our datset will be a text file were each entry is a patient's case followed by the following sentences depending on their availability:
 1. the first sentence of the therapist's message.
 2. the sentence in the which either of these keywords appear:
   1. seems like
   2. sounds like
   3. feels like

## Developement
---
We will begin by importing the needed libraries, mounting Google Drive (where the original data resides and the processed data will be written to), and defining some helper functions.

In [0]:
import pandas as pd
import string
from tqdm.notebook import tqdm

In [0]:
def clean_text(text):
    text = str(text)
    text = text.replace("\n", " ").replace("\t", " ").replace("  ", " ").strip()
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = text + "."
    return text

In [0]:
def first_sentence(text):
    text = str(text)
    period = text.find(".") + 1
    text = text[:period]
    return text

In [0]:
def sentence_with_key_words(text, key_words):
    text = str(text)
    text_lower = text.lower()
    for k in key_words:
        pos = text_lower.find(k)
        if pos > 0:
            return text[pos : text.find(".", pos)]
    return None

In [0]:
def dataset_maker(df, q_col, ans_col, key_words, file_out):
    processed_data = ""
    for i, row in tqdm(df.iterrows(), total=len(df), desc="Processing data"):
        q = clean_text(row[q_col])
        ans = clean_text(first_sentence(row[ans_col]))
        processed_data += q + ans + "\n\n"
        key_search = sentence_with_key_words(row[ans_col], key_words)
        if key_search is not None:
            processed_data += q + clean_text(key_search)

    with open(file_out, "w") as f:
        f.write(processed_data)

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
df = pd.read_csv("../gdrive/My Drive/counsel_chat.csv")

## Results
---
Now that we have developed all we needed we can simply call the `dataset_maker` and it will produce our dataset.

In [14]:
input_datamaker = {
    "df": df,
    "q_col": "questionText",
    "ans_col": "answerText",
    "key_words": ["seems like", "sounds like", "feels like"],
    "file_out": "../gdrive/My Drive/councelchat.txt",
}
dataset_maker(**input_datamaker)